# Create and test metadata_handler

In [15]:
import src.metadata_handler as mdh

%autoreload 2

In [16]:
from iptcinfo import IPTCInfo
from iptcinfo import IPTCData
import json

from iptcinfo import c_datasets, c_datasets_r

import os
import pandas as pd
import numpy as np

import boto3

In [25]:
a_photo = 'first_sample/EK000025-2.JPG'
info = IPTCInfo(a_photo)
data_dict = dict(info.data)
info_field_nums = set(data_dict.keys())
test_set = set(['a', 'b', 'c'])

('WARNING: problems with charset recognition', "'\\x1b'")


In [265]:
# rework get_fields to 
def get_fields(directory):
    ## strip '/' only if it is at the end
    #  otherwise it may mess with nested dirs
    #  (e.g. sample/photo/set)
    # directory = directory.strip('/')
    field_set = set()
    for p, dirs, files in os.walk(directory):
        for ff in files:
            if ff[-4:] == '.JPG':
                info = IPTCInfo(directory+'/'+ ff)
                data_dict = dict(info.data)
                fields = set(data_dict.keys())
                field_set = field_set.union(fields)                
    return field_set


In [276]:
# fields = set([])
# fields = get_fields('first_sample/')
# fields

# SQL direct from photo?

Signature: info.exportSQL(tablename, mappings, extra)
Docstring:
statement = info.exportSQL('mytable', mappings, extra-data)

mappings = {
    'IPTC dataset name here': 'your table column name here',
    'caption/abstract': 'caption',
    'city': 'city',
    'province/state': 'state} # etc etc etc.

Returns a SQL statement to insert into your given table name a set
of values from the image. Caller passes in a reference to a hash
which maps IPTC dataset names into column names for the database
table. Optionally pass in a ref to a hash of extra data which will
also be included in the insert statement. Keys in that hash must
be valid column names.

### Create SQL table

```
<table name>
    |
    |- <col name = field name>
    |- ...
    |- ...
    |
    |- <some way to reference/retrieve photo
```

### Use IPTCInfo to import metadata

```python
for photo in photos:
    info = IPTCInfo(photo)
    info.exportSQL('<table name>', map{})
```    

# JSON from metadata

In [154]:
def set_to_dict(field_set):
    field_dict = {}
    for field in field_set:
        field_dict[field] = c_datasets[field]
    return field_dict

In [277]:
# field_dict = set_to_dict(fields)
# field_dict

In [216]:
field_dict[999] = 'fake field'

def Mdata_dictionary(photo):
    ''' get a photos metadata as dict '''

    # return relevant data for a photo
    dd = {}
    info = IPTCInfo(photo)
    data_dict = dict(info.data)         
    for field in field_dict.keys():
        if data_dict.has_key(field):
            # print field_dict[field],' : ', data_dict[field]
            dd[field_dict[field]] = data_dict[field]
        else:
            print field
            # dd[field_dict[k]] = None
    print dd
    return dd

def Mdata_to_json(photo, outfile):
    with open(outfile, 'a') as outf:
        json.dump(Mdata_dictionary(photo), outf, indent = 4)


In [268]:
# Mdata_to_json('first_sample/EK000004.JPG', 'test.json')

In [269]:
# field_dict

# Data Base!

In [273]:
db = pd.read_json('data/test_DB.json')

In [275]:
db.head().T

,0,1,2,3,4
by-line,Cathy Clark,Picasa,Cathy Clark,Picasa,Cathy Clark
caption/abstract,M2E32L107-108R390B311,NaN,1,NaN,1
contact,[],[],[],[],[]
copyright notice,Conservation Northwest,NaN,Conservation Northwest,NaN,Conservation Northwest
date created,2.01505e+07,NaN,2.01506e+07,NaN,2.01201e+07
file_path,first_sample/EK000004-2.JPG,first_sample/EK000004.JPG,first_sample/EK000010-2.JPG,first_sample/EK000010.JPG,first_sample/EK000011-2.JPG
keywords,[Camera Check],[camera check],[hoary marmot],[marmot],[hoary marmot]
object name,wolverine,NaN,wolverine,NaN,wolverine
sub-location,Blackjack Ridge-2015-1,NaN,Blackjack Ridge-2015-1,NaN,Blackjack Ridge-2015-1
supplemental category,[],[],[],[],[]


# AWS and boto3 conversion

In [17]:
# http://docs.aws.amazon.com/AmazonS3/latest/dev/UsingBucket.html#access-bucket-intro

s3 = boto3.resource('s3')
photo_bucket = s3.Bucket('cnwphotos')


```python
s3_client = boto3.client('s3')
open('hello.txt').write('Hello, world!')

# Upload the file to S3
s3_client.upload_file('hello.txt', 'MyBucket', 'hello-remote.txt')

# Download the file from S3
s3_client.download_file('MyBucket', 'hello-remote.txt', 'hello2.txt')
print(open('hello2.txt').read())

```

In [20]:
for bucket in s3.buckets.all():
    print(bucket.name)
    
# # Upload a new file
# data = open('test.jpg', 'rb')
# s3.Bucket('my-bucket').put_object(Key='test.jpg', Body=data)

# size = sum(1 for _ in photo_bucket.objects.all())
# size

photo_list = list(photo_bucket.objects.limit(10))
    

In [24]:
photo_list[0].get()

{u'AcceptRanges': 'bytes',
 u'Body': <botocore.response.StreamingBody at 0x117654b50>,
 u'ContentLength': 466275,
 u'ContentType': 'image/jpeg',
 u'ETag': '"64a6a8a6fe9f6c21f85f87df64d1281c"',
 u'LastModified': datetime.datetime(2016, 11, 4, 4, 59, 48, tzinfo=tzutc()),
 u'Metadata': {},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '466275',
   'content-type': 'image/jpeg',
   'date': 'Thu, 10 Nov 2016 01:01:32 GMT',
   'etag': '"64a6a8a6fe9f6c21f85f87df64d1281c"',
   'last-modified': 'Fri, 04 Nov 2016 04:59:48 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'z9bTM88d4o8bH/M/WcuKPtODG/oCQ8X7N81mQo/BcnZKpFS3nFSaZpyb3KYLKFruEJ8aBuY+FnY=',
   'x-amz-request-id': '31E7DF83BE6BFFCA'},
  'HTTPStatusCode': 200,
  'HostId': 'z9bTM88d4o8bH/M/WcuKPtODG/oCQ8X7N81mQo/BcnZKpFS3nFSaZpyb3KYLKFruEJ8aBuY+FnY=',
  'RequestId': '31E7DF83BE6BFFCA',
  'RetryAttempts': 0}}

In [22]:
# mdh.build_dictionary(photo_list[0].get())

In [23]:
client = s3.meta.client

In [ ]:
client.download_file('cnwphotos', )